## Projects
2.- How does SARS-Cov-02 infection alter transcripyional acitivity. Overall design: Comparative gene expression profiling analysis of RNA-seq data using 4 types (uninfected/infected, young/mid-aged) of isolated lung endothelial cells.

 SRAID      | Age    |  Treatment |
------------|--------|------------|
SRR24206824 | Young  | SARS-CoV-2
SRR24206825 | MidAge | SARS-CoV-2
SRR24206826 | Young  | PBS
SRR24206827 | MidAge | PBS

In [ ]:
# Make sure the following is installed in conda
  # SRA Toolkit
  # FastQC
  # Cutadapt
  # Hisat2
  # Subread
# Get appropriate reference genome (mouse)
    !wget ftp://ftp.ccb.jhu.edu/pub/infphilo/hisat2/data/mm10.tar.gz
    !tar -xzf mm10.tar.gz
# Get appropriate annotation genome (GTF/GFF)
    !wget https://ftp.ensembl.org/pub/release-102/gtf/mus_musculus/Mus_musculus.GRCm38.102.gtf.gz
    !gunzip Mus_musculus.GRCm38.102.gtf.gz
# Download Samples and ensure proper name/nomenclature
  # Subsample the data to 5 M reads
    !fastq-dump -X 5000000 SRR24206824
    !fastq-dump -X 5000000 SRR24206825
    !fastq-dump -X 5000000 SRR24206826
    !fastq-dump -X 5000000 SRR24206827
    # Rename files to reflect experimental conditions
      !mv SRR24206824.fastq young_infected.fastq
      !mv SRR24206825.fastq midage_infected.fastq
      !mv SRR24206826.fastq young_control.fastq
      !mv SRR24206827.fastq midage_control.fastq

  # FastQC and assess
      !fastqc *.fastq
  # Cutadapt (15 and 28) - Industry standard
    !cutadapt --cut 210 -q 28 --minimum-length 50 -o trimmed_young_infected.fastq young_infected.fastq #option one, which gives 100% unpaired
    !cutadapt --cut 25 -q 28 -o trimmed_midage_infected.fastq midage_infected.fastq #default option, which gives 100% unpaired
    !cutadapt --cut 25 -q 28 -o trimmed_young_control.fastq young_control.fastq
    !cutadapt --cut 25 -q 28 -o trimmed_midage_control.fastq midage_control.fastq
  # Hisat alignment with proper annotation
      # alignment
      !hisat2 -x mm10/genome -U trimmed_young_infected.fastq -S aln_young_infected.sam
      !hisat2 -x mm10/genome -U trimmed_midage_infected.fastq -S aln_midage_infected.sam
      !hisat2 -x mm10/genome -U trimmed_young_control.fastq -S aln_young_control.sam
      !hisat2 -x mm10/genome -U trimmed_midage_control.fastq -S aln_midage_control.sam
  # Subread to create counts table
    !featureCounts -a Mus_musculus.GRCm38.102.gtf -t exon -g gene_id -o counts.txt *.sam
    !head -n 50 counts.txt
# Once data is processed, visualize with R
  # Mutate function to sum all counts per row
    # Which gene has the most counts?
    # Length of the longest and shorted gene?
  # Visualize counts across all samples with ggplot (histogram)

In [ ]:
#library(tidyverse)
#library(DESeq2)

# Read counts
#count_data <- read.table("counts.txt", header=TRUE, row.names=1, skip=1)

# Basic analysis
#total_counts <- colSums(count_data[,6:ncol(count_data)])
#gene_lengths <- count_data$Length

# Visualization
#ggplot(data.frame(Sample=names(total_counts), Counts=total_counts),
       #aes(x=Sample, y=Counts)) +
  #geom_bar(stat="identity") +
  #theme_minimal()